# Detekcija i rešavanje osmosmerke

In [ ]:
from PIL import Image
from pytesseract import Output
import cv2
import pytesseract
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = 25, 15

import import_ipynb
from wordsearcher import find_matches

In [ ]:
# poboljšanje rezolucije slike na 300 DPI
def change_resolution(image_path):
    image = Image.open(image_path)
    image.save(image_path, dpi=(300, 300))
    
# učitavanje slike
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

# konvertovanje slike u grayscale
def image_to_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# konvertovanje slike u binarnu
def image_to_bin(image_gs):
    height, width = image_gs.shape[0:2]
    image_binary = np.ndarray((height, width), dtype=np.uint8)
    return cv2.threshold(image_gs, 127, 255, cv2.THRESH_BINARY)[1]

# ispravljanje rotirane slike
def skew_correction(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]
    # cv2.minAreaRect() vraća vrednosti iz intervala [-90, 0)
    if angle == 0:
        return image
    elif angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_REPLICATE)

# uklanjanje šumova
def smoothing_image(image):
    t1 = cv2.threshold(image, 180, 255, cv2.THRESH_BINARY)[1]
    t2 = cv2.threshold(t1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    blur = cv2.medianBlur(t2, 1, 0)
    t3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return t3

# otvaranje = erozija + dilacija
def opening(image):
    kernel = np.ones((3, 3))
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

# prikaz slike
def display_image(image, color=False):
    if color:
        plt.imshow(image)
    else:
        plt.imshow(image, 'gray')

# dobavljanje okvira osmosmerke
def find_frame(contours, half_height):
    contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
    for i in range(1, len(contours)):
        frame_contour = contours[i]
        x, y, w, h = cv2.boundingRect(frame_contour)
        if y < half_height:
            return (x, y, w, h)

# selektovanje regiona oko slova zadatih reči
def select_letters(image_bin, contours, frame):
    mask = np.zeros(image_bin.shape[:2], dtype=np.uint8)
    sorted_regions = [] 
    regions_array = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour) 
        if h < 100 and h > 20:
            # frame[0] je x koordinata gornjeg levog ćoška frame-a, frame[1] je y koordinata, a frame[3] je visina frame-a
            if x < frame[0] or y > frame[1] + frame[3]:
                cv2.rectangle(mask, (x, y), (x + w, y + h), (255, 0, 0), -1)
    return mask        

# isecanje slike na veličinu frame-a
def crop_to_frame(image, frame):
    x, y, w, h = frame
    return image[y:y+h+1, x:x+w+1]

# dobavljanje sortiranih površina unutar kontura
def get_sorted_areas(contours):
    areas = []
    for contour in contours:
        areas.append(cv2.contourArea(contour))
    areas.sort()
    return areas

# uklanjanje slike unutar osmosmerke, ukoliko ona postoji
def remove_image(wordsearch, contours):
    #print(get_sorted_areas(contours))
    contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
    max_area = cv2.contourArea(contours[0])
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour) 
        area = cv2.contourArea(contour)
        if area < max_area / 2:
            cv2.rectangle(wordsearch, (x - 10, y - 10), (x + w + 10, y + h + 10), (255, 255, 255), -1)
            return

# uklanjanje crnog okvira oko osmosmerke
def remove_black_frame(image):
    x, y, w, h = 0, 0, image.shape[1], image.shape[0]
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 255), 30)

# primena OCR-a za zadate reči
def ocr_words(image):
    d = pytesseract.image_to_data(image, lang='srp_latn', config='--oem 1', output_type=Output.DICT)
    n_boxes = len(d['text'])
    results = []
    confidences = []
    for i in range(n_boxes):
        if d['conf'][i] != '-1':
            results.append(d['text'][i])
            confidences.append(float(d['conf'][i]))
    
    return d, results, round(sum(confidences) / len(confidences), 2)

# primena OCR-a za osmosmerku
def ocr_wordsearch(image):
    d = pytesseract.image_to_data(image, lang='srp_latn', config='--psm 6 --oem 0', output_type=Output.DICT)
    n_boxes = len(d['text'])
    results = []
    confidences = []
    prev_ln = '0'
    for i in range(n_boxes):
        result = d['text'][i]
        confidence = d['conf'][i]
        ln = d['line_num'][i]
        
        if confidence != '-1':
            if ln == prev_ln:
                results[-1] += result
            else:
                results.append(result)
                prev_ln = ln
            confidences.append(float(confidence))
    
    return d, results, round(sum(confidences) / len(confidences), 2)

## Pretprocesiranje slika

In [ ]:
# TODO: odraditi za sve slike
change_resolution('data/10.jpg')
image_color = load_image('data/10.jpg')
image_rotated = skew_correction(image_color)
image_bin = image_to_bin(image_to_gray(image_rotated))
image_smooth = smoothing_image(image_bin)
image = opening(image_smooth)
#cv2.imwrite('data/3.jpg', image)
    
display_image(image)

## Detekcija zadatih reči

In [ ]:
# prvo pronalazimo okvir osmosmerke, kako bi posmatrali samo reči izvan njega
contours = cv2.findContours(image.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
frame = find_frame(contours, image.shape[0] / 2)
x, y, w, h = frame
cv2.rectangle(image_color, (x, y), (x + w, y + h), (255, 0, 0), -1)

# izolujemo samo regione koji sadrže zadate reči
mask = select_letters(image, contours, frame) 
image_words = cv2.bitwise_and(image, image, mask=mask)
image_words[mask==0] = 255

display_image(image_words)

In [ ]:
dict_words, words, confidence = ocr_words(image_words)

print("Prepoznate reči: ", words)
print("Procenat sigurnosti: ", confidence)

## Detekcija slova u osmosmerci

In [ ]:
# isecanje dela slike koji sadrži osmosmerku
image_wordsearch = crop_to_frame(image, frame)
img = crop_to_frame(image_color, frame)
contours = cv2.findContours(image_wordsearch.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]

# uklanjanje slike rešenja unutar osmosmerke
remove_image(image_wordsearch, contours)

# uklanjanje crnog okvira oko osmosmerke
remove_black_frame(image_wordsearch)

display_image(image_wordsearch)

In [ ]:
dict_wordsearch, wordsearch, confidence = ocr_wordsearch(image_wordsearch)

print("Prepoznata slova (linije): ", wordsearch)
print("Procenat sigurnosti: ", confidence)

## Rešavanje osmosmerke

In [ ]:
matches = find_matches(wordsearch, words)
print(len(words))
print(len(matches))

## Prikaz i analiza dobijenog rešenja